# Natural Language Processing
- **Week 1**
  Tokenized, sequenced and padded words to prepare a sentiment analysis. Every word is given an index. Words of a sentence are combined in a tokenized representation array. To unify the different sequence sizes we applied a padding of zeroes, such as allowing tensorflow to work with the texts. 
- **Week 2**
  Used embeddings and tensorflow datasets to make a sentiment analysis of IMDB reviews. Instead of using the previously applied *word-bag-model* we represent words in an n-dimensional vector space as so called *word embedding*. The weights of the word embedding can be visualized in a 3d projection using [tensorflow projector](https://projector.tensorflow.org).
- **Week 3**
  Applied a *Recurrent Neural Network* (RNN) and used *long short-term memory* (LSTM) besides *gated recurrent unit* (GRU). RNN can be applied to sequences as sentences. Here previously seen words can have an impact of the meaning of the sentence. RNN permit us to transfer this information to later calculations in the sequence. LSTM and GRU further introduces a internal memory state to deal with long term temporal dependencies.

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O ./assets/bbc-text.csv \
    -nv

2019-11-07 11:41:09 URL:https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv [5057493/5057493] -> "./assets/bbc-text.csv" [1]


In [1]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(tf.__version__)

2.0.0


In [15]:
sentences = []
with open('./assets/bbc-text.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        sentences += row[1:]
print(len(sentences))
print(sentences[0])

2225
tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to 

In [18]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokens = tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

print(len(word_index))

29727


In [22]:
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')
print(padded[0])
print(padded.shape)

[177 265   7 ...   0   0   0]
(2225, 4491)


## Week2

### import data

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [7]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Hyperparameters

In [8]:
vocab_size = 10000
oov_token = '<OOV>'
max_length = 120
trunc_type = 'post'
embedding_dim = 16
num_epochs = 10

# Tokenization

In [9]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [26]:
word_index = tokenizer.word_index
reverse_word_index = dict([(val, index) for (index, val) in word_index.items()])

### Model

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final),
          verbose=2)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10


25000/25000 - 3s - loss: 0.4938 - acc: 0.7484 - val_loss: 0.3450 - val_acc: 0.8505


Epoch 2/10


25000/25000 - 2s - loss: 0.2421 - acc: 0.9066 - val_loss: 0.3832 - val_acc: 0.8330


Epoch 3/10


...

25000/25000 - 3s - loss: 8.2560e-04 - acc: 1.0000 - val_loss: 0.7540 - val_acc: 0.8196


Epoch 9/10


25000/25000 - 3s - loss: 3.3887e-04 - acc: 1.0000 - val_loss: 0.7896 - val_acc: 0.8214


Epoch 10/10


25000/25000 - 2s - loss: 1.7957e-04 - acc: 1.0000 - val_loss: 0.8233 - val_acc: 0.8219


### Export - Get Embedding weights and write to file

In [32]:
e = model.layers[0]
weights = e.get_weights()[0]

import io

out_v = io.open('./assets/imdb_vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('./assets/imdb_meta.tsv', 'w', encoding='utf-8')

for i in range(1, vocab_size):
    embeddings = weights[i]
    word = reverse_word_index[i]
    out_m.write(word + '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n') 

## Week 3 - RNN: Recurrent Neural Networks

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64), return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    # tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               41472     
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 202,253
Trainable params: 202,253
Non-trainable params: 0
_________________________________________________________________


In [56]:
embedding_dim = 100
max_length = 16
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_size = 160000
test_portion = .1

In [39]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O .assets/training_cleaned.csv

--2019-11-08 16:00:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 

216.58.211.48
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.211.48|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

          /tmp/trai   0%[                    ]       0  --.-KB/s               

         /tmp/train   0%[                    ] 216.23K  1012KB/s               

...

                 /t  99%[==================> ] 225.69M  1.85MB/s    eta 2s     

                /tm  99%[==================> ] 226.32M  1.96MB/s    eta 1s     

               /tmp  99%[==================> ] 226.85M  2.03MB/s    eta 1s     

              /tmp/  99%[==================> ] 227.44M  2.11MB/s    eta 1s     

/tmp/training_clean 100%[===================>] 227.87M  2.15MB/s    in 1m 44s  

2019-11-08 16:02:05 (2.19 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [82]:
import csv
corpus = []

with open('./assets/training_cleaned.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        list_item = [row[5]]
        label = row[0]
        if label == '0':
            list_item.append(0)
        else:
            list_item.append(1)
        corpus.append(list_item) 

In [102]:
import random
random.shuffle(corpus)

labels = []
sentences = []
for x in range(1, training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])

tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index)
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

split = int(test_portion * training_size)

test_sequences = padded[:split]
test_labels = np.array(labels[:split])
training_sequences = padded[split:]
training_labels = np.array(labels[split:])

## Download and prepare GloVe: Global Vectors for Word Representation

In [86]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O ./assets/glove.6B.100d.txt \
    -nv

2019-11-08 16:42:22 URL:https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt [347116733/347116733] -> "./assets/glove.6B.100d.txt" [1]


In [89]:
embeddings_index = {};
with open('./assets/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector; 

In [96]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 16, 100)           13844600  
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 100)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 16, 128)           84480     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 64)                41216     
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [101]:
print(type(training_labels))
training_labels_final = np.array(training_labels)
print(type(training_labels_final))

<class 'list'>
<class 'numpy.ndarray'>


In [103]:
num_epochs = 10
history = model.fit(training_sequences, training_labels, epochs=num_epochs, verbose=2)

Train on 143999 samples
Epoch 1/10


InvalidArgumentError:  indices[10,0] = 138460 is not in [0, 138446)
	 [[node sequential_3/embedding_11/embedding_lookup (defined at /usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_227574]

Function call stack:
distributed_function
